# STEP 1: Read the adjacency matrix, derive the CTMC and save to CSV
##What This Code Does
- Loads the adjacency matrix (milan_sensitive_hex_adjacency.csv)
- Loads the real hex grid (milan_sensitive_hexes.geojson)
- Ensures all hex IDs in the adjacency matrix are present in the hex grid
- For each adjacent pair, it generates transitions between relevant states:
    - Normal → Compromised
    - Compromised → Down

Down → Mitigated

Mitigated → Recovered

Recovered → Normal



In [1]:
import math
import os
from pathlib import Path
import pandas as pd

"""CTMC generator (absorbing version)

*   Assumes the numerical entries in the *_rates dictionaries are **per‑hour
    transition probabilities**.
*   Converts them to **exponential rates λ (h⁻¹)** on the fly using
        λ = −ln(1 − p) / Δt   with Δt = 1 h.
*   Treats the state **Recovered** as *absorbing* (no outward transitions).

Outputs one CSV per infrastructure class in ``csv/ctmcs`` with columns::

    source,target,rate,type

where ``source`` / ``target`` are strings like ``H42|Compromised`` and
``type`` ∈ {internal, spatial}.
"""

# === Helper ================================================================

DT_HOURS = 1.0  # time slice (hours)


def prob_to_rate(p: float, dt: float = DT_HOURS) -> float | None:
    """Convert a per‑interval probability **p** to an exponential rate λ.

    Returns ``None`` for p ≥ 1, which signals an absorbing self‑loop that
    should *not* be written as an explicit transition in the generator.
    """
    if p <= 0.0:
        return 0.0
    if p >= 1.0:
        return None  # absorbing / invalid as a rate
    return -math.log1p(-p) / dt  # ln(1 − p)


# === Default transition *probabilities* (generic) ==========================
initial_probs = {
    "Normal":      {"Normal": 0.98, "Compromised": 0.02},
    "Compromised": {"Compromised": 0.95, "Down": 0.05},
    "Down":        {"Down": 0.95, "Mitigated": 0.15},
    "Mitigated":   {"Mitigated": 0.95, "Recovered": 0.05},
    # Recovered is absorbing → 100 % stay; handled implicitly
    "Recovered":   {"Recovered": 1.0},
}

# === Class‑specific transition *probabilities* ============================
class_probs = {
    "power": {
        "Normal":      {"Normal": 0.97, "Compromised": 0.03},
        "Compromised": {"Compromised": 0.90, "Down": 0.10},
        "Down":        {"Down": 0.92, "Mitigated": 0.08},
        "Mitigated":   {"Mitigated": 0.93, "Recovered": 0.07},
        "Recovered":   {"Recovered": 1.0},
    },
    "telecom": {
        "Normal":      {"Normal": 0.97, "Compromised": 0.03},
        "Compromised": {"Compromised": 0.93, "Down": 0.07},
        "Down":        {"Down": 0.94, "Mitigated": 0.06},
        "Mitigated":   {"Mitigated": 0.94, "Recovered": 0.06},
        "Recovered":   {"Recovered": 1.0},
    },
    "hospital": {
        "Normal":      {"Normal": 0.95, "Compromised": 0.05},
        "Compromised": {"Compromised": 0.85, "Down": 0.15},
        "Down":        {"Down": 0.90, "Mitigated": 0.10},
        "Mitigated":   {"Mitigated": 0.90, "Recovered": 0.10},
        "Recovered":   {"Recovered": 1.0},
    },
    "ambulance_station": {
        "Normal":      {"Normal": 0.95, "Compromised": 0.05},
        "Compromised": {"Compromised": 0.87, "Down": 0.13},
        "Down":        {"Down": 0.90, "Mitigated": 0.10},
        "Mitigated":   {"Mitigated": 0.90, "Recovered": 0.10},
        "Recovered":   {"Recovered": 1.0},
    },
    "surveillance": {
        "Normal":      {"Normal": 0.98, "Compromised": 0.02},
        "Compromised": {"Compromised": 0.95, "Down": 0.05},
        "Down":        {"Down": 0.95, "Mitigated": 0.05},
        "Mitigated":   {"Mitigated": 0.95, "Recovered": 0.05},
        "Recovered":   {"Recovered": 1.0},
    },
    "gov_office": {
        "Normal":      {"Normal": 0.96, "Compromised": 0.04},
        "Compromised": {"Compromised": 0.90, "Down": 0.10},
        "Down":        {"Down": 0.92, "Mitigated": 0.08},
        "Mitigated":   {"Mitigated": 0.92, "Recovered": 0.08},
        "Recovered":   {"Recovered": 1.0},
    },
    "bank": {
        "Normal":      {"Normal": 0.96, "Compromised": 0.04},
        "Compromised": {"Compromised": 0.91, "Down": 0.09},
        "Down":        {"Down": 0.93, "Mitigated": 0.07},
        "Mitigated":   {"Mitigated": 0.93, "Recovered": 0.07},
        "Recovered":   {"Recovered": 1.0},
    },
    "atm": {
        "Normal":      {"Normal": 0.96, "Compromised": 0.04},
        "Compromised": {"Compromised": 0.92, "Down": 0.08},
        "Down":        {"Down": 0.94, "Mitigated": 0.06},
        "Mitigated":   {"Mitigated": 0.94, "Recovered": 0.06},
        "Recovered":   {"Recovered": 1.0},
    },
}

# === Spatial propagation rates (already λ in h⁻¹) =========================
spatial_rates = {
    "hospital": 0.005,
    "power": 0.02,
    "telecom": 0.015,
    "ambulance_station": 0.01,
    "surveillance": 0.008,
    "gov_office": 0.007,
    "bank": 0.005,
    "atm": 0.004,
}

# === Paths ==============================================================
BASE_DIR = Path(".")
input_folder = BASE_DIR / "csv" / "adjacency"
output_folder = BASE_DIR / "csv" / "ctmcs"
output_folder.mkdir(parents=True, exist_ok=True)

# === CTMC generation =====================================================
for file in os.listdir(input_folder):
    if not file.endswith(".csv"):
        continue

    filepath = input_folder / file
    class_name = file.replace("adjacency_", "").replace(".csv", "")
    print(f"Processing class: {class_name} from {file}")

    probs = class_probs.get(class_name, initial_probs)
    spatial_rate = spatial_rates.get(class_name, 0.01)  # already a rate

    # --- Load & relabel adjacency matrix ---------------------------------
    adj_matrix = pd.read_csv(filepath, index_col=0)
    hexes = adj_matrix.index.tolist()
    hex_id_map = {h: f"H{i}" for i, h in enumerate(hexes)}
    adj_matrix.index = [hex_id_map[h] for h in adj_matrix.index]
    adj_matrix.columns = [hex_id_map[h] for h in adj_matrix.columns]

    transitions: list[dict] = []

    # --- Internal transitions -------------------------------------------
    for h in adj_matrix.index:
        for src_state, targets in probs.items():
            for dst_state, p in targets.items():
                if src_state == dst_state:
                    # self-loop stays implicit in the diagonal; skip writing it
                    continue
                λ = prob_to_rate(p)
                if λ is None or λ == 0.0:
                    # absorbing or zero-rate edge
                    continue
                transitions.append({
                    "source": f"{h}|{src_state}",
                    "target": f"{h}|{dst_state}",
                    "rate": λ,
                    "type": "internal",
                })


        # --- Spatial transitions (Compromised → Compromised) ------------
        for neighbor, connected in adj_matrix.loc[h].items():
            if connected:
                transitions.append({
                    "source": f"{h}|Compromised",
                    "target": f"{neighbor}|Compromised",
                    "rate": spatial_rate,
                    "type": "spatial",
                })

    # --- Write CSV -------------------------------------------------------
    output_path = output_folder / f"ctmc_{class_name}.csv"
    pd.DataFrame(transitions).to_csv(output_path, index=False)
    print(f" Saved CTMC to {output_path}")


Processing class: ambulance_station from adjacency_ambulance_station.csv
 Saved CTMC to csv\ctmcs\ctmc_ambulance_station.csv
Processing class: atm from adjacency_atm.csv
 Saved CTMC to csv\ctmcs\ctmc_atm.csv
Processing class: bank from adjacency_bank.csv
 Saved CTMC to csv\ctmcs\ctmc_bank.csv
Processing class: gov_office from adjacency_gov_office.csv
 Saved CTMC to csv\ctmcs\ctmc_gov_office.csv
Processing class: hospital from adjacency_hospital.csv
 Saved CTMC to csv\ctmcs\ctmc_hospital.csv
Processing class: power from adjacency_power.csv
 Saved CTMC to csv\ctmcs\ctmc_power.csv
Processing class: surveillance from adjacency_surveillance.csv
 Saved CTMC to csv\ctmcs\ctmc_surveillance.csv
Processing class: telecom from adjacency_telecom.csv
 Saved CTMC to csv\ctmcs\ctmc_telecom.csv
Processing class: h3_matrix from h3_adjacency_matrix.csv
 Saved CTMC to csv\ctmcs\ctmc_h3_matrix.csv
